In [1]:
import os, pandas, numpy, systole, systole.reports, bokeh

ibi = []
for f in [f for f in os.listdir () if f[-4:].lower () == '.csv' and f != 'current.csv']:
    df = pandas.read_csv (f, usecols = ['simple', 'regular', 'peaks'], dtype = numpy.float64).dropna ()
    df.attrs['name'] = f[:-4]
    ibi.append (df)
if (len (ibi) < 1):
    raise SystemExit ("No CSV files found.")
ibi = pandas.concat (ibi, keys = [df.attrs['name'] for df in ibi])
ibi.index.names = ["Source", "Index"]
source_indexes = ibi.index.get_level_values ("Source").unique ()

def report_domain (callable):
    tds = pandas.concat ([callable (ibi['regular'][i]) for i in source_indexes], keys = source_indexes)
    tds.index.names = ["Source", "Index"]
    return pandas.pivot_table (tds, index = "Source", values = "Values", columns = "Metric")

def show_plot (callable, cols = 1, sizing_mode = None, width = None, **kwargs):
    ps = [callable (ibi['regular'][i], input_type = "rr_ms", backend = "bokeh", **kwargs) for i in source_indexes]
    for i, p in enumerate (ps):
        p.title.text = source_indexes[i]
    bokeh.plotting.show (bokeh.layouts.gridplot (ps, sizing_mode = sizing_mode, width = width, ncols = cols))

bokeh.io.output_notebook ()

Loading BokehJS ...

In [10]:
ibi_artefacts = []
for d in [systole.detection.rr_artefacts (ibi['simple'][i]) for i in source_indexes]:
    ibi_artefacts.append (pandas.DataFrame.from_dict (
        {k : [(d[k] == True).sum ()] for k in ['ectopic', 'short', 'long', 'missed', 'extra']}))
pandas.concat (ibi_artefacts, keys = source_indexes)


,,ectopic,short,long,missed,extra
Source,,,,,,
202304142052_dinner_unwell,0,19,2,24,9,0
202304151136_driving,0,68,2,64,8,0


In [3]:
report_domain (systole.hrv.time_domain)

Metric,MaxBPM,MaxRR,MeanBPM,MeanRR,MedianBPM,MedianRR,MinBPM,MinRR,RMSSD,SDNN,SDSD,nn50,pnn50
Source,,,,,,,,,,,,,
202304142052_dinner_unwell,112.570356,1032.0,82.072417,737.40705,81.743869,734.0,58.139535,533.0,27.008545,68.54755,27.010680,428.0,6.766798
202304151136_driving,114.068441,996.0,82.691999,730.63033,82.417582,728.0,60.240964,526.0,23.155099,60.91584,23.155657,756.0,3.646713


In [4]:
show_plot (systole.plots.plot_rr, sizing_mode = "scale_width", show_artefacts = False, slider = False, figsize = 200)

In [5]:
report_domain (systole.hrv.frequency_domain)

Metric,hf_peak,hf_power,hf_power_nu,hf_power_per,lf_hf_ratio,lf_peak,lf_power,lf_power_nu,lf_power_per,total_power,vlf_peak,vlf_power,vlf_power_per
Source,,,,,,,,,,,,,
202304142052_dinner_unwell,0.152344,343.320625,19.516486,10.118864,4.123873,0.093750,1415.810727,80.483514,41.728910,3392.877295,0.003906,1633.745943,48.152226
202304151136_driving,0.152344,269.482138,21.023677,11.878877,3.756542,0.050781,1012.320907,78.976323,44.623497,2268.582638,0.003906,986.779593,43.497626


In [6]:
show_plot (systole.plots.plot_frequency, cols = 4, width = 400, figsize = 200)

In [7]:
report_domain (systole.hrv.nonlinear_domain)

Metric,SD1,SD2,determinism_rate,l_max,l_mean,recurrence_rate,shannon_entropy
Source,,,,,,,
202304142052_dinner_unwell,19.097925,95.033241,76.533891,5266.0,14.388963,37.73813,3.470915
202304151136_driving,16.373127,84.575658,76.637181,16316.0,16.248938,41.27693,3.607407


In [8]:
show_plot (systole.plots.plot_poincare, cols = 4, width = 400, figsize = 400)

In [9]:
show_plot (systole.plots.plot_ectopic , cols = 4, figsize = 400)
show_plot (systole.plots.plot_shortlong , cols = 4, figsize = 400)